In [75]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters=250
dropout = 0.2

cuda


In [19]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [5]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([82, 46, 63, 60,  1, 42, 73, 70, 65, 60, 58, 75,  1, 33, 76, 75, 60, 69,
        57, 60, 73, 62,  1, 60, 28, 70, 70, 66,  1, 70, 61,  1, 30, 70, 73, 70,
        75, 63, 80,  1, 56, 69, 59,  1, 75, 63, 60,  1, 49, 64, 81, 56, 73, 59,
         1, 64, 69,  1, 41, 81,  0,  1,  1,  1,  1,  0, 46, 63, 64, 74,  1, 60,
        57, 70, 70, 66,  1, 64, 74,  1, 61, 70, 73,  1, 75, 63, 60,  1, 76, 74,
        60,  1, 70, 61,  1, 56, 69, 80, 70, 69])


In [55]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[75, 63, 80,  1, 76, 71, 70, 69],
        [12,  1, 35, 69,  0, 75, 63, 64],
        [60, 73,  1, 73, 76, 61, 61, 67],
        [73, 60, 66, 56,  1, 66, 69, 70]], device='cuda:0')
targets:
tensor([[63, 80,  1, 76, 71, 70, 69,  1],
        [ 1, 35, 69,  0, 75, 63, 64, 74],
        [73,  1, 73, 76, 61, 61, 67, 60],
        [60, 66, 56,  1, 66, 69, 70, 58]], device='cuda:0')


In [35]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits,loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index,None)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index,index_next),dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


fAPI_K﻿'O2'X*ckm1d&[﻿&n1.I&-Q3B﻿IA(saxa5J?vRT*g"bS!j.Pk[[AMCF6WlIG"l/*#2Z69696r_OQuXsa.KmDRn/smfAu7SWu(WYAvYZ6pIbBDRCtqoHzA/mL﻿yR/NBE!uOfssmVt6VX2﻿jgpHW.v46_y4PCu;QaDVdpCVNb0B!i&n
U1V4emx5!d2cJ#F*vQC8iE0NjBJ;9DIGNQqQVN1lQiu(ko#13Smt;s5_aOAIQVl[.﻿pI#6 hFcHW8(droohCuMA GL/
dX'VOf.n﻿e;V[-G8#VdxT8#s7U-P2O..2ZCrzTqau?&'Qdan.mes]UWP&6mm'Zk #"mN4.KFwFc/Q[x1_pg-iF/N(,etdt'67FEJUtk0 v_xdsJhRCXO2tMSHKrb4uF*rA(:YQq
QM1Pn/h'DDyza06lJV﻿(.*]0NLv4udaymmD:p?rT8icgF7O_Cmzw;#J6s-SgC5gz﻿B(/NN/-M!i]5tHj,smeCiM1x5J8


In [73]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logists,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [81]:
 # pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    # sample a batch of data
    if iter % eval_iters==0:
        losses = estimate_loss()
        print(f'step:{iter}, train_loss:{losses['train']:.4f}, val_loss:{losses['val']:.4f}')
    xb, yb = get_batch('train')
    #evaluate the loss
    logits, loss = model.forward(xb,yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step:0, train_loss:2.4383, val_loss:2.4787
step:250, train_loss:2.4342, val_loss:2.4902
step:500, train_loss:2.4526, val_loss:2.4721
step:750, train_loss:2.4357, val_loss:2.4743
step:1000, train_loss:2.4273, val_loss:2.4966
step:1250, train_loss:2.4223, val_loss:2.5012
step:1500, train_loss:2.4486, val_loss:2.4872
step:1750, train_loss:2.4209, val_loss:2.4915
step:2000, train_loss:2.4436, val_loss:2.4724
step:2250, train_loss:2.4251, val_loss:2.4966
step:2500, train_loss:2.4236, val_loss:2.4776
step:2750, train_loss:2.4383, val_loss:2.4612
step:3000, train_loss:2.4472, val_loss:2.5070
step:3250, train_loss:2.4257, val_loss:2.4703
step:3500, train_loss:2.3862, val_loss:2.5005
step:3750, train_loss:2.4490, val_loss:2.4787
step:4000, train_loss:2.4198, val_loss:2.4638
step:4250, train_loss:2.4219, val_loss:2.4850
step:4500, train_loss:2.4249, val_loss:2.4744
step:4750, train_loss:2.4273, val_loss:2.5135
step:5000, train_loss:2.4453, val_loss:2.4952
step:5250, train_loss:2.4211, val_loss:2

In [59]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)



theantekisurd.
heathislogras ave usur aided d sme orshaveslks ttur  burese-pait. iot d r t d hale t
ara d adnkinand ben bie sas, ath set,"
e o inqugo, agashoond finr
s are erdiacat asiventh dom. than f s ileothemeed uicanoss I id memeke im Theve heront she I't Jinsar w we

m cctizariclarcarg kid mac.
inorenthugsod bung b. a," sigap ad waut?" r m asoy the th't yo gg Wileiened us,  h haceed Westes; I n tun]

caded Jid bu urcofes aineaithe tilecafing aft igoure aificain'thyharingranduthy ine s ime


In [83]:
x = torch.tensor([-0.05],dtype=torch.float32)
y = nn.ReLU(x)
print(y)

ReLU(inplace=True)
